# Gear Design

### Note this is for design of 20 degree full depth gear only

#### User Input 

In [23]:
P = 20*(10**3) #Power in W
Np = 3000 #Pinion rotation speed in RPM(Revolutions Per Minute)
i = 2 #Gear ratio
Sut = 600 #Ultimate Tensitle Strength of gear in N/mm^2
Ep = 200*(10**3)#Youngs Modulus of Pinion in MPa
Eg = 200*(10**3)#Youngs Modulus of Gear in MPa
Grade = 5 #Gear Grade (Accuracy of gear made)
Cs = 2 #Service Factor
fs = 2 #Factor of Safety

#### Assumption

In [24]:
zp = 18 #Minimum number of Teeth for Pinion Assumed
zg = i*zp #Number of teeth on Gear based on Gear ratio

#### Lewis form factor Data

In [25]:
import pandas as pd
Lewis_path = "D:/Gear Design Project Data/Lewis Form Factor.csv"
L = pd.read_csv(Lewis_path)

In [26]:
L

,Z,Y
0,12,0.245
1,13,0.261
2,14,0.277
3,15,0.290
4,16,0.296
5,17,0.303
6,18,0.309
7,19,0.314
8,20,0.322
9,21,0.328


#### Getting Lewis form factor for pinion from above table

In [27]:
Y = float(L[L.Z ==str(zp)]["Y"])
Y

0.309

#### Defining Lewis Equation

In [28]:
from sympy import pi, diff, lambdify, symbols
m = symbols('m',real = True)
dp = m*zp
v = pi*m*zp*Np/60000
Cv = 3/(3+v) #Change if velocity is higher
L = (P/v)*(Cs/Cv)*fs
b = 10*m
sigmab = Sut/3
R = m*b*sigmab*Y
f = L-R
f_dm = diff(f,m)

In [29]:
f

-618.0*m**2 + 400000*(3*pi*m/5 + 2)/(9*pi*m)

In [30]:
f_dm

-1236.0*m + 80000/(3*m) - 400000*(3*pi*m/5 + 2)/(9*pi*m**2)

In [31]:
f_l = lambdify(m,f,'numpy')
f_dm_l = lambdify(m,f_dm,'numpy')
v_l = lambdify(m,v,'numpy')

#### Define Function to calculate module for given Initial guess using Newton Rhapson Method

In [32]:
def CalcModule(m):
    f0 = f_l(m)
    mn = m
    
    while not abs(f0)<(10*(10**-10)):
        fx = f_l(mn)
        fdx = f_dm_l(mn)
        mn = m - fx/fdx
        m = mn
        f0 = fx
    
    return mn

In [33]:
m0 = 10
m = CalcModule(m0)
print('First Module according to calculation is {one}'.format(one=round(m,4))+" mm")

First Module according to calculation is 7.0461 mm


#### List of Preffered Modules

In [34]:
m_list = [0.1,0.2,0.3,0.5,0.6,0.8,1,1.25,1.5,2,2.5,3,4,5,6,8,10,12,16,20,25,32,40,50]

#### Calculates Preffered Module

In [35]:
for n in range(len(m_list)):
    m_diff = m_list[n]-m
    if m_diff>=0:
        ind = n
        break

mf = m_list[ind]
print('Prefered Module is',mf,"mm")

Prefered Module is 8 mm


In [36]:
v = v_l(mf)
print('Velocity of gear is ',round(v,4),"m/sec")

Velocity of gear is  22.6195 m/sec


#### Calculating Actual loading with Buckingham Equation

#### Database for Gear error

In [37]:
E = pd.read_csv("D:/Gear Design Project Data/Gear Error.csv")
E

,Grade,b0,b1
0,1,0.80,0.06
1,2,1.25,0.10
2,3,2.00,0.16
3,4,3.20,0.25
4,5,5.00,0.40
5,6,8.00,0.63
6,7,11.00,0.90
7,8,16.00,1.25
8,9,22.00,1.80
9,10,32.00,2.50


In [38]:
b0 = float(E[E.Grade==Grade]["b0"])
b1 = float(E[E.Grade==Grade]["b1"])

#### Calculating  Gear Errors

In [39]:
def Buckingham(m):
    #for Pinion
    dp = m*zp
    phip = m + 0.25*(dp**0.5)
    ep = b0 + b1*phip

    #for gear
    dg = m*zg
    phig = m + 0.25*(dg**0.5)
    eg = b0 + b1*phig

    #total
    e = (ep+eg)*(10**-3)
    
    k = 0.111 # Constant for 20 degree full depth Gear
    C = k/((1/Ep)+(1/Eg)) # Deformation Factor in N/mm^2
    b = 10*m # Face width of gears
    
    v = v_l(m)
    Pt = P/v
    a = 21*v
    d = C*e*b + Pt
    Pd = (a*d)/(a+(d**0.5))
    Ptotal = Pt*Cs + Pd
    
    Sb = m*b*(Sut/3)*Y
    
    fs_a = Sb/Ptotal

    return Ptotal,fs_a

#### Calculation of Total Load using Buckingham equation and Factor of Safety

In [40]:
Ptotal, fs_a = Buckingham(mf)

while fs_a<fs:
    mf = m_list[ind+1]
    ind = ind+1
    Ptotal , fs_a = Buckingham(mf)
        
print("Total Load is {one}".format(one = round(Ptotal,4)))
print("Factor of Safety with Dynamic Load is {one}".format(one = round(fs_a,4)))
print("Module seleced for gear is {one}".format(one=mf)+" mm")

Total Load is 15817.9358
Factor of Safety with Dynamic Load is 2.5005
Module seleced for gear is 8 mm


#### Calculating Brinnel Hardness Number with Wear equation

In [41]:
#Sb = b*Q*Dp*k
b =10*mf
dp = mf*zp
Q = (2*zg)/(zp+zg)
k = Ptotal*fs/(b*Q*dp)
BHN = 100*(k/0.16)**0.5
print("Brinnel Hardness Number for gear should be {one}".format(one=round(BHN,2)))

Brinnel Hardness Number for gear should be 358.79


# Summarizing all design Parameters

In [42]:
b = 10*mf
v = v_l(mf)
print("Number of teeth on pinion is {one}".format(one=zp))
print("Number of teeth on gear is {one}".format(one=zg))
print("Module selected for gear is {one}".format(one=mf)+" mm")
print("Face width of gear is {one}".format(one=b)+" mm")
print("Velocity of gears is {one}".format(one=round(v,4))+" m/sec")
print("Total Load on gear is {one}".format(one=round(Ptotal,4))+" N")
print("Actual Factor of Safety in Bending Strength is {one}".format(one=round(fs_a,4)))
print("Brinnel Hardness number for gear should be {one}".format(one=round(BHN,2)))

Number of teeth on pinion is 18
Number of teeth on gear is 36
Module selected for gear is 8 mm
Face width of gear is 80 mm
Velocity of gears is 22.6195 m/sec
Total Load on gear is 15817.9358 N
Actual Factor of Safety in Bending Strength is 2.5005
Brinnel Hardness number for gear should be 358.79
